In [1]:
import numpy as np
import pandas as pd

In [3]:
!pip install openpyxl

In [5]:
# statistical tests TET2

df = pd.read_excel(r"D:\Users\albao\Downloads\TET2_predictions_annotationsall_char.xlsx")


In [9]:
list(df.columns)


['Unnamed: 0',
 'SAMPLE',
 'FILTER',
 'VAR_ID',
 'CHR',
 'POS',
 'REF',
 'ALT',
 'GENE',
 'ALLELE',
 'VARIANT_CLASS',
 'BIOTYPE',
 'TYPE',
 'FEATURE',
 'EXON',
 'HGVSc',
 'HGVSp',
 'cDNA_position',
 'CDS_position',
 'Protein_position',
 'Amino_acids',
 'Codons',
 'STRAND',
 'REFSEQ_MATCH',
 'GIVEN_REF',
 'USED_REF',
 'BAM_EDIT',
 'IMPACT',
 'CONSEQUENCE',
 'SOMATIC',
 'PHENO',
 'CLIN_SIG',
 'COSMIC_MATCH',
 'HEURISTIC',
 'PREVIOUSLY_IDENTIFIED',
 'WHITELIST',
 'ARTIFACT',
 'SIFT_DESC',
 'SIFT_SCORE',
 'POLYPHEN_DESC',
 'POLYPHEN_SCORE',
 'CADD_PHRED',
 'CADD_RAW',
 'REF_DEPTH_VIS1',
 'ALT_DEPTH_VIS1',
 'F1R2_VIS1',
 'F2R1_VIS1',
 'REF_DEPTH_VIS3',
 'ALT_DEPTH_VIS3',
 'F1R2_VIS3',
 'F2R1_VIS3',
 'VAF_VIS1',
 'VAF_VIS3',
 'VAF_ABSOLUTA',
 'VAF_RELATIVA',
 'VAF_RATIO',
 'CHIP_driver',
 'FU_yrs',
 'AER',
 'log_positive_AER',
 'log_AER',
 'N',
 'doubling_time',
 'doubling_time_wo_outliers',
 'log_ratio_VAF_years',
 'VAF_absolute_yrs',
 'growth_lm_VAF',
 'growth_lm_log_VAF',
 'fitness_exp',


In [24]:
df["CpG variant"].unique()

array(['Not CPG variant', 'removed', 'added'], dtype=object)

In [25]:
import warnings
import pandas as pd
from scipy import stats

ignore = ['SAMPLE','FILTER','VAR_ID','CHR','REF','ALT','GENE', 'ALLELE','VARIANT_CLASS','BIOTYPE','TYPE','HGVSc','HGVSp','EXON','cDNA_position',
 'CDS_position','Amino_acids','Codons','Protein_position', 'GIVEN_REF','USED_REF','BAM_EDIT','IMPACT','CONSEQUENCE','SOMATIC',
 'PHENO','CLIN_SIG','COSMIC_MATCH','HEURISTIC','PREVIOUSLY_IDENTIFIED','WHITELIST','ARTIFACT','SIFT_DESC','SIFT_SCORE','POLYPHEN_DESC',
 'POLYPHEN_SCORE','CADD_PHRED','CADD_RAW','REF_DEPTH_VIS1','ALT_DEPTH_VIS1','F1R2_VIS1','F2R1_VIS1','REF_DEPTH_VIS3',
 'ALT_DEPTH_VIS3','F1R2_VIS3','F2R1_VIS3','VAF_VIS1','VAF_VIS3','VAF_ABSOLUTA','VAF_RELATIVA','VAF_RATIO','CHIP_driver',
 'FU_yrs','AER','log_positive_AER','log_AER','N','doubling_time','doubling_time_wo_outliers','log_ratio_VAF_years','VAF_absolute_yrs','growth_lm_VAF',
 'growth_lm_log_VAF','fitness_exp','region_normal','region_alt','POS.1',"FEATURE", "STRAND", "IMPACT.1", 'VARIANT_CLASS.1',]
df["CpG variant"] = df["CpG variant"].map({'Not CPG variant':0, 'removed':-1, 'added':1})
warnings.filterwarnings("ignore")

tf = pd.DataFrame()
for item in df.columns:
    print(f"'{item}',")
    if item in ignore:
        continue
    positive = df[df["CHIP_driver"] == "YES"][item]
    negative = df[df["CHIP_driver"] == "NO"][item]
    
    # shapiro wilk drivers
    _, pos_shapiro_p = stats.shapiro(positive)
    _, neg_shapiro_p = stats.shapiro(negative)
    
    # shapiro wilk non drivers
    _, levene_p = stats.levene(positive, negative)
    
    # if normal distribution for both use t test, else use a mannwhitney test
    if pos_shapiro_p > 0.05 and neg_shapiro_p > 0.05 and levene_p > 0.05:
        _, ttest_p = stats.ttest_ind(positive, negative, equal_var=True)
        test_type = "t_test"
        test_p = ttest_p
    else:
        _, mannwhitney_p = stats.mannwhitneyu(positive, negative, alternative='two-sided')
        test_type = "mannwhitney"
        test_p = mannwhitney_p
        tf[item] = [pos_shapiro_p, neg_shapiro_p, levene_p, test_type, test_p]

tf.index = ["shapiro_p_chip", "shapiro_p_nonchip", "levene_p_value", "test_used", "test_p_value"]

tf = tf.transpose()
print(tf)
tf.to_excel("./statistics.xlsx")

'Unnamed: 0',
'SAMPLE',
'FILTER',
'VAR_ID',
'CHR',
'POS',
'REF',
'ALT',
'GENE',
'ALLELE',
'VARIANT_CLASS',
'BIOTYPE',
'TYPE',
'FEATURE',
'EXON',
'HGVSc',
'HGVSp',
'cDNA_position',
'CDS_position',
'Protein_position',
'Amino_acids',
'Codons',
'STRAND',
'REFSEQ_MATCH',
'GIVEN_REF',
'USED_REF',
'BAM_EDIT',
'IMPACT',
'CONSEQUENCE',
'SOMATIC',
'PHENO',
'CLIN_SIG',
'COSMIC_MATCH',
'HEURISTIC',
'PREVIOUSLY_IDENTIFIED',
'WHITELIST',
'ARTIFACT',
'SIFT_DESC',
'SIFT_SCORE',
'POLYPHEN_DESC',
'POLYPHEN_SCORE',
'CADD_PHRED',
'CADD_RAW',
'REF_DEPTH_VIS1',
'ALT_DEPTH_VIS1',
'F1R2_VIS1',
'F2R1_VIS1',
'REF_DEPTH_VIS3',
'ALT_DEPTH_VIS3',
'F1R2_VIS3',
'F2R1_VIS3',
'VAF_VIS1',
'VAF_VIS3',
'VAF_ABSOLUTA',
'VAF_RELATIVA',
'VAF_RATIO',
'CHIP_driver',
'FU_yrs',
'AER',
'log_positive_AER',
'log_AER',
'N',
'doubling_time',
'doubling_time_wo_outliers',
'log_ratio_VAF_years',
'VAF_absolute_yrs',
'growth_lm_VAF',
'growth_lm_log_VAF',
'fitness_exp',
'region_normal',
'region_alt',
'POS.1',
'exon',
'Amino_acids.1',
'IMP

In [67]:
tf.to_excel("./importances_tf.xlsx")

In [68]:
import warnings
import pandas as pd
from scipy import stats

warnings.filterwarnings("ignore")

# Initialize the DataFrame to store the results
tf = pd.DataFrame()

# Loop through each column in tf_columns
for item in tf_columns:
    positive = df[df["binarized prediction"] == 1][item]
    negative = df[df["binarized prediction"] == 0][item]
    
    # Perform Shapiro-Wilk test
    _, pos_shapiro_p = stats.shapiro(positive)
    _, neg_shapiro_p = stats.shapiro(negative)
    
    # Perform Mann-Whitney U test
    _, mannwhitney_p = stats.mannwhitneyu(positive, negative, alternative='two-sided')
    
    # Store the results in the DataFrame
    tf[item] = [pos_shapiro_p, neg_shapiro_p, mannwhitney_p]

# Rename the rows for clarity
tf.index = ["shapiro_p_chip", "shapiro_p_nonchip", "mannwhitney_p_value"]

# Transpose the DataFrame to have items as columns and tests as rows
tf = tf.transpose()

# Display the DataFrame
print(tf)


               shapiro_p_chip  shapiro_p_nonchip  mannwhitney_p_value
USF2_score       1.000000e+00       1.029730e-31         5.939461e-08
RNF2_score       1.000000e+00       1.431137e-31         2.613368e-08
REST_score       1.000000e+00       1.029730e-31         5.939461e-08
RAD21_score      1.000000e+00       3.865148e-34         5.928925e-03
BRD4_score       1.000000e+00       3.523085e-35         2.206701e-01
...                       ...                ...                  ...
RUNX3_score      1.000000e+00       7.130173e-35         8.267866e-02
BHLHE40_score    1.000000e+00       3.865148e-34         5.928925e-03
SIN3A_score      1.000000e+00       1.431137e-31         2.613368e-08
H3K4Me1          2.704870e-21       1.718032e-24         3.005362e-01
H3K4Me3          2.093050e-17       7.648820e-24         2.490026e-01

[61 rows x 3 columns]


In [69]:
tf.to_excel("./importances_tf_model_classifications.xlsx")